In [14]:
#============================================================================================
# Classifier Training
# Author: Gerald M (Changed by Maksim L)
#
# This script uses a convolution neural network classifier, to train for cells and non-cell
# objects.
#============================================================================================

import os
import random
import shutil
import datetime
from multiprocessing import cpu_count
import skimage.exposure

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop
import tensorflow as tf

os.environ['MKL_NUM_THREADS'] = str(cpu_count())
os.environ['GOTO_NUM_THREADS'] = str(cpu_count())
os.environ['OMP_NUM_THREADS'] = str(cpu_count())
os.environ['openmp'] = 'True'

config = tf.ConfigProto(device_count={"GPU" : 1, "CPU" : cpu_count()})
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

def AHE(img):
    img_adapteq = skimage.exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq

#=============================================================================================
# Precheck on directory structure
#=============================================================================================

if os.listdir('8-bit/test_data/cell/'):
    print 'Warning! Files detected in test data directory!'
    print 'Moving files back to training data directory...'

    for f in os.listdir('8-bit/test_data/cell/'):
        shutil.move('8-bit/test_data/cell/'+f,'8-bit/training_data/cell/'+f)

    for f in os.listdir('8-bit/test_data/nocell/'):
        shutil.move('8-bit/test_data/nocell/'+f,'8-bit/training_data/nocell/'+f)
        
if os.listdir('8-bit/validation_data/cell/'):
    print 'Warning! Files detected in validation data directory!'
    print 'Moving files back to training data directory...'

    for f in os.listdir('8-bit/validation_data/cell/'):
        shutil.move('8-bit/validation_data/cell/'+f,'8-bit/training_data/cell/'+f)

    for f in os.listdir('8-bit/validation_data/nocell/'):
        shutil.move('8-bit/validation_data/nocell/'+f,'8-bit/training_data/nocell/'+f)
        


cell_data = os.listdir('8-bit/training_data/cell/')
nocell_data = os.listdir('8-bit/training_data/nocell/')

test_cell_data = random.sample(cell_data, int(0.1*len(cell_data)))
test_nocell_data = random.sample(nocell_data, int(0.1*len(nocell_data)))

for f in test_cell_data:
    shutil.move('8-bit/training_data/cell/'+f,'8-bit/test_data/cell/'+f)

for f in test_nocell_data:
    shutil.move('8-bit/training_data/nocell/'+f,'8-bit/test_data/nocell/'+f)    

Warning! Files detected in test data directory!
Moving files back to training data directory...


In [6]:

#=============================================================================================
# Construction of Convolution Neural Network
#=============================================================================================

print "Constructing Convolution Neural Network..."

# Create object of sequential class
classifier = Sequential()

# Convolution operation
# Number of filters
# Shape of each filter
# Input shape in first two, input type in third, 3 = RGB
# activation function, relu = rectifier function
classifier.add(Conv2D(8, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))

# Pooling operation
# Pooling reduces size of images in order to reduce number of nodes
# 2x2 matrix
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Batch normalization
# Normalize the activations of the previous layer at each batch
# i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.
classifier.add(BatchNormalization())

classifier.add(Conv2D(16, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(BatchNormalization())

classifier.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(BatchNormalization())

classifier.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(BatchNormalization())

# Prevent nodes from generating same classifiers
classifier.add(Dropout(0.5))

# Flattening operation
# Convert pooled images into vectors

classifier.add(Flatten())

# Connect the set of nodes which input to connected layers
# Units is number of nodes
classifier.add(Dense(units = 128, activation = 'relu'))

# Prevent nodes from generating same classifiers
classifier.add(Dropout(0.5))

# Initialise output layer
classifier.add(Dense(units = 1, activation = 'sigmoid')) #softmax

# Compile
# Optimizer is stochastic gradient descent
# Loss is loss function
# Metric is performance metric
optimizer = RMSprop(lr=1e-4)
classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.summary()

print "Done!\n"

Constructing Convolution Neural Network...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 78, 78, 8)         80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 39, 39, 8)         0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 39, 39, 8)         32        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 37, 37, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 18, 18, 16)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 18, 18, 16)        64        
_________________________________________________________________
conv2d_7 (Conv2D)            (Non

In [10]:
#=============================================================================================
# Preparing training and test data
#=============================================================================================

from keras.preprocessing.image import ImageDataGenerator

print "Splitting all training data into 70% training and 30% test data directories..."

cell_data = os.listdir('8-bit/training_data/cell/')
nocell_data = os.listdir('8-bit/training_data/nocell/')

validation_cell_data = random.sample(cell_data, int(0.3*len(cell_data)))
validation_nocell_data = random.sample(nocell_data, int(0.3*len(nocell_data)))

for f in validation_cell_data:
    shutil.move('8-bit/training_data/cell/'+f,'8-bit/validation_data/cell/'+f)

for f in validation_nocell_data:
    shutil.move('8-bit/training_data/nocell/'+f,'8-bit/validation_data/nocell/'+f)

# training data
train_datagen = ImageDataGenerator(rotation_range=180, rescale = 1./255, shear_range = 0.15, zoom_range = 0.15, width_shift_range=0.15, height_shift_range=0.15, horizontal_flip = True, vertical_flip = True)
training_data = train_datagen.flow_from_directory('8-bit/training_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale')
#training_data = train_datagen.flow_from_directory('8-bit/training_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale', save_to_dir='preview', save_prefix='cell', save_format='jpeg')
# validation data
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_data = validation_datagen.flow_from_directory('8-bit/validation_data', target_size = (80, 80), batch_size = 32, class_mode = 'binary', color_mode = 'grayscale')

print "Done!\n"


Splitting all training data into 70% training and 30% test data directories...
Found 8821 images belonging to 2 classes.
Found 9181 images belonging to 3 classes.
Done!



In [9]:

#=============================================================================================
# Fitting model to data
#=============================================================================================

print "Fitting model to data..."

# Find number of epoch and validation steps
steps_epoch = len([filename for filename in os.listdir('8-bit/training_data/cell') if filename.endswith(".tif")]) + len([filename for filename in os.listdir('8-bit/training_data/nocell') if filename.endswith(".tif")])
steps_valid = len([filename for filename in os.listdir('8-bit/validation_data/cell') if filename.endswith(".tif")]) + len([filename for filename in os.listdir('8-bit/validation_data/nocell') if filename.endswith(".tif")])

# Checkpoint to only save the best model, metric = val_acc
filepath = "cc_model_"+datetime.datetime.today().strftime('%Y_%m_%d')+".h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# steps_per_epoch is number of images in training set
history = classifier.fit_generator(training_data, steps_per_epoch = steps_epoch, epochs = 50, callbacks=callbacks_list, validation_data = validation_data, validation_steps = steps_valid, shuffle = True)

print "Done!\n"

Fitting model to data...
Epoch 1/50
12602/12602 [==============================] - 749s 59ms/step - loss: 0.1186 - acc: 0.9585 - val_loss: -2.0012 - val_acc: 0.0169

Epoch 00001: val_acc improved from -inf to 0.01685, saving model to cc_model_2019_01_26.h5
Epoch 2/50
11059/12602 [=========================>....] - ETA: 1:19 - loss: 0.0776 - acc: 0.9744

KeyboardInterrupt: 

In [1]:
#=============================================================================================
# Plotting accuracy
#=============================================================================================

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('Acc.png')
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('Loss.png')

NameError: name 'history' is not defined

In [ ]:
# Finally evaluating model on the test data

test_generator = test_datagen.flow_from_directory(
'8-bit/test_data',
target_size=(80, 80),
batch_size=32,
class_mode='binary',
color_mode = 'grayscale')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)